In [1]:
import string
import numpy as np
from scipy.stats import norm
from statsmodels import robust
import pandas as pd
from IPython.display import display
pd.set_option('max_rows', 5)

# 外れ値・異常値・欠損値の除去・補完

## 外れ値・異常値・欠損値とは
---
<table class="text-left border">
    <tr>
        <th class="border-bottom border-right-bold">外れ値 (outlier)</th>
        <td>他の値から大きく外れた値</td>
        <td><li>特異なイベントの発生</li></td>
    </tr>
    <tr>
        <th class="border-bottom border-right-bold">異常値 (anomaly)</th>
        <td>データの生成プロセスに照らして、起こるべきでない異常によって生じた値<br />異常値である外れ値と異常値でない外れ値は<strong>データのみでは区別できない</strong></td>
        <td><li>センサーの故障</li><li>データの入力ミス</li><li>表計算ソフトの集計行混入</li></td>
    </tr>
    <tr>
        <th class="border-bottom border-right-bold">欠損値 (missing value)</th>
        <td>値の抜けているデータ</td>
        <td><li>NULL 値</li><li>空欄</li><li>プレースホルダー文字列</li></td>
    </tr>
</table>

外れ値・異常値の検出は中級編以降で扱い、ここでは欠損値の確認、検出した外れ値・欠損値の除去・補完を扱う。

## 欠損値

### 確認
---
`pandas.DataFrame.isnull`を使用する。

In [2]:
n_sample = 1000
n_features = 5
np.random.seed(1234)
sample = pd.DataFrame(
    np.random.normal(size=(n_sample, n_features)),
    columns=[string.ascii_uppercase[i] for i in range(n_features)])
for i, rate in enumerate(np.linspace(0, 0.5, n_features)):
    missing = np.random.choice(n_sample,
                               size=int(n_sample * rate),
                               replace=False)
    sample.iloc[missing, i] = np.nan

print('sample')
display(sample)

sample


,A,B,C,D,E
0,0.471435,-1.190976,1.432707,NaN,-0.720589
1,0.887163,0.859588,-0.636524,0.015696,NaN
...,...,...,...,...,...
998,-0.442212,-1.350463,NaN,0.842840,NaN
999,-0.592877,0.708934,1.608824,1.995282,0.033269


In [3]:
help(pd.DataFrame.isna)

Help on function isna in module pandas.core.frame:

isna(self) -> 'DataFrame'
    Detect missing values.
    
    Return a boolean same-sized object indicating if the values are NA.
    NA values, such as None or :attr:`numpy.NaN`, gets mapped to True
    values.
    Everything else gets mapped to False values. Characters such as empty
    strings ``''`` or :attr:`numpy.inf` are not considered NA values
    (unless you set ``pandas.options.mode.use_inf_as_na = True``).
    
    Returns
    -------
    DataFrame
        Mask of bool values for each element in DataFrame that
        indicates whether an element is not an NA value.
    
    See Also
    --------
    DataFrame.isnull : Alias of isna.
    DataFrame.notna : Boolean inverse of isna.
    DataFrame.dropna : Omit axes labels with missing values.
    isna : Top-level isna.
    
    Examples
    --------
    Show which entries in a DataFrame are NA.
    
    >>> df = pd.DataFrame({'age': [5, 6, np.NaN],
    ...                    

In [4]:
sample.isna()

,A,B,C,D,E
0,False,False,False,True,False
1,False,False,False,False,True
...,...,...,...,...,...
998,False,False,True,False,True
999,False,False,False,False,False


列ごとに欠損値の有無を確かめるには`any`、欠損値の数を確かめるには`sum`を使用する。  
`pandas.DataFrame.info`を使用してもよい。

In [5]:
sample.isna().any()

A    False
B     True
C     True
D     True
E     True
dtype: bool

In [6]:
sample.isna().sum()

A      0
B    125
C    250
D    375
E    500
dtype: int64

In [7]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A       1000 non-null   float64
 1   B       875 non-null    float64
 2   C       750 non-null    float64
 3   D       625 non-null    float64
 4   E       500 non-null    float64
dtypes: float64(5)
memory usage: 39.2 KB


### 除去
---
最も単純な対処法だが、やりすぎるとデータが不足したり、必要な情報まで失われる可能性がある。  
欠損がランダムに発生するわけではない場合 (年齢の高い人や収入の高い人ほど収入を尋ねる欄に回答しないなど) は、**取り除いてしまうとデータに偏りが生じる**。

`pandas.DataFrame.dropna`を使用する。

In [8]:
help(pd.DataFrame.dropna)

Help on function dropna in module pandas.core.frame:

dropna(self, axis=0, how='any', thresh=None, subset=None, inplace=False)
    Remove missing values.
    
    See the :ref:`User Guide <missing_data>` for more on which values are
    considered missing, and how to work with missing data.
    
    Parameters
    ----------
    axis : {0 or 'index', 1 or 'columns'}, default 0
        Determine if rows or columns which contain missing values are
        removed.
    
        * 0, or 'index' : Drop rows which contain missing values.
        * 1, or 'columns' : Drop columns which contain missing value.
    
        .. versionchanged:: 1.0.0
    
           Pass tuple or list to drop on multiple axes.
           Only a single axis is allowed.
    
    how : {'any', 'all'}, default 'any'
        Determine if row or column is removed from DataFrame, when we have
        at least one NA or all NA.
    
        * 'any' : If any NA values are present, drop that row or column.
        * 'all' :

#### 欠損を含む行

In [9]:
sample.dropna()

,A,B,C,D,E
2,1.150036,0.991946,0.953324,-2.021255,-0.334077
7,-0.122092,0.124713,-0.322795,0.841675,2.390961
...,...,...,...,...,...
997,0.275159,0.075078,1.361621,-1.096261,-1.101649
999,-0.592877,0.708934,1.608824,1.995282,0.033269


#### 欠損を含む列

In [10]:
sample.dropna(axis=1)

,A
0,0.471435
1,0.887163
...,...
998,-0.442212
999,-0.592877


#### 全ての値が欠損している行

In [11]:
sample.loc[:, 'B':'E'].dropna(how='all')

,B,C,D,E
0,-1.190976,1.432707,NaN,-0.720589
1,0.859588,-0.636524,0.015696,NaN
...,...,...,...,...
998,-1.350463,NaN,0.842840,NaN
999,0.708934,1.608824,1.995282,0.033269


#### 欠損でない値が任意の数以上ある行を残す

In [12]:
sample.dropna(thresh=3)

,A,B,C,D,E
0,0.471435,-1.190976,1.432707,NaN,-0.720589
1,0.887163,0.859588,-0.636524,0.015696,NaN
...,...,...,...,...,...
998,-0.442212,-1.350463,NaN,0.842840,NaN
999,-0.592877,0.708934,1.608824,1.995282,0.033269


#### 特定の列に欠損がある行

In [13]:
sample.dropna(subset=['B'])

,A,B,C,D,E
0,0.471435,-1.190976,1.432707,NaN,-0.720589
1,0.887163,0.859588,-0.636524,0.015696,NaN
...,...,...,...,...,...
998,-0.442212,-1.350463,NaN,0.842840,NaN
999,-0.592877,0.708934,1.608824,1.995282,0.033269


### 補完
---
欠損値を別の値で埋めること。  
ここでは代表値による補完を扱い、より高度な補完手法は中級編以降で扱う。

欠損値の補完には、`pandas.DataFrame.fillna`を使用する。

In [14]:
help(pd.DataFrame.fillna)

Help on function fillna in module pandas.core.frame:

fillna(self, value=None, method=None, axis=None, inplace=False, limit=None, downcast=None) -> Union[ForwardRef('DataFrame'), NoneType]
    Fill NA/NaN values using the specified method.
    
    Parameters
    ----------
    value : scalar, dict, Series, or DataFrame
        Value to use to fill holes (e.g. 0), alternately a
        dict/Series/DataFrame of values specifying which value to use for
        each index (for a Series) or column (for a DataFrame).  Values not
        in the dict/Series/DataFrame will not be filled. This value cannot
        be a list.
    method : {'backfill', 'bfill', 'pad', 'ffill', None}, default None
        Method to use for filling holes in reindexed Series
        pad / ffill: propagate last valid observation forward to next valid
        backfill / bfill: use next valid observation to fill gap.
    axis : {0 or 'index', 1 or 'columns'}
        Axis along which to fill missing values.
    inplace 

#### 平均値での補完。

In [15]:
sample.fillna(sample.mean())

,A,B,C,D,E
0,0.471435,-1.190976,1.432707,-0.018202,-0.720589
1,0.887163,0.859588,-0.636524,0.015696,-0.004376
...,...,...,...,...,...
998,-0.442212,-1.350463,0.064951,0.842840,-0.004376
999,-0.592877,0.708934,1.608824,1.995282,0.033269


#### 中央値での補完。

In [16]:
sample.fillna(sample.median())

,A,B,C,D,E
0,0.471435,-1.190976,1.432707,-0.045519,-0.720589
1,0.887163,0.859588,-0.636524,0.015696,0.019233
...,...,...,...,...,...
998,-0.442212,-1.350463,0.050845,0.842840,0.019233
999,-0.592877,0.708934,1.608824,1.995282,0.033269


## 外れ値
---
条件式などを使って絞り込む。

### 3$\sigma$法
---
平均から$\pm 3\sigma $ (標準偏差) を超える値を外れ値とする方法。正規分布では約 $0.03\%$ が外れ値になる。

In [17]:
mean = sample['A'].mean()
sigma = sample['A'].std(ddof=0)
sample.loc[(mean - 3 * sigma <= sample['A']) &
           (sample['A'] <= mean + 3 * sigma)]

,A,B,C,D,E
0,0.471435,-1.190976,1.432707,NaN,-0.720589
1,0.887163,0.859588,-0.636524,0.015696,NaN
...,...,...,...,...,...
998,-0.442212,-1.350463,NaN,0.842840,NaN
999,-0.592877,0.708934,1.608824,1.995282,0.033269


練習問題

---
正規分布で$\pm 3\sigma$範囲に含まれる値の割合を求める。

解答例

---

In [18]:
norm.cdf(3) - norm.cdf(-3)

0.9973002039367398

---

### Hampel判別法 (Hampel identifier)
---
$3\sigma$法の平均を中央値に、標準偏差を中央絶対偏差 (Median Absolute Deviation) に置き換えた方法。$3\sigma$法よりサンプルに含まれる外れ値の影響を受けにくい。

$\displaystyle median( x) \pm 3\times ( MAD\times 1.4826)$

$\displaystyle MAD=median( |x_{i} -median( x) |)$

$1.4826$ は正規分布を仮定した場合に、中央絶対偏差 ($75\%$ 点に相当) を標準偏差に補正するための定数。

練習問題

---
中央絶対偏差を求める以下の関数 mad を完成させる。

In [19]:
def mad(x):
    return

解答例

---

In [20]:
def mad(x):
    return np.median(np.abs(x - np.median(x)))

---

In [21]:
mad(sample['A'])

0.6558910693556668

中央絶対偏差は`statsmodels.robust.mad`で求められる。  
既定値では、正規化定数 $c=0.6744897501960817$ で割って補正してあるので、そのままの値を求めるには $c=1$ とする。 (正規化定数で割った値を中央絶対偏差と呼ぶこともある)

In [22]:
help(robust.mad)

Help on function mad in module statsmodels.robust.scale:

mad(a, c=0.6744897501960817, axis=0, center=<function median at 0x7f10040854c0>)
    The Median Absolute Deviation along given axis of an array
    
    Parameters
    ----------
    a : array_like
        Input array.
    c : float, optional
        The normalization constant.  Defined as scipy.stats.norm.ppf(3/4.),
        which is approximately .6745.
    axis : int, optional
        The default is 0. Can also be None.
    center : callable or float
        If a callable is provided, such as the default `np.median` then it
        is expected to be called center(a). The axis argument will be applied
        via np.apply_over_axes. Otherwise, provide a float.
    
    Returns
    -------
    mad : float
        `mad` = median(abs(`a` - center))/`c`



In [23]:
robust.mad(sample['A'], c=1)

0.6558910693556668

練習問題

---
標準正規分布の $75\%$ 点を求め、 $0.6744897501960817$ と比較する。  
$1$ を標準正規分布の $75\%$ 点で割り、 $1.4826$ と比較する。

解答例

---

In [24]:
print(norm.ppf(0.75))
print(1 / norm.ppf(0.75))

0.6744897501960817
1.482602218505602


---

練習問題

---
Hampel 判別法に基づいて外れ値かどうかを判定する (外れ値でない場合に True 、外れ値の場合に False を返す) 以下の関数 Hampel を完成させる。

In [25]:
def hampel(ndarray):
    return

解答例

---

In [26]:
def hampel(ndarray):
    med = np.median(ndarray)
    mad = robust.mad(ndarray)
    return ((med - 3 * mad) <= ndarray) & (ndarray <= (med + 3 * mad))

---

In [27]:
sample.loc[hampel(sample['A'])]

,A,B,C,D,E
0,0.471435,-1.190976,1.432707,NaN,-0.720589
1,0.887163,0.859588,-0.636524,0.015696,NaN
...,...,...,...,...,...
998,-0.442212,-1.350463,NaN,0.842840,NaN
999,-0.592877,0.708934,1.608824,1.995282,0.033269


### 箱ひげ図
---
箱ひげ図では第 1 四分位点 ($Q_{1/4}$, lower quartile) 、第 3 四分位点 ($Q_{3/4}$, upper quartile) 、四分位範囲 (IQR, interquartile range) を用いて、以下のように外れ値を決めることが多い。

$Q_{1/4} -1.5*IQR,Q_{3/4} +1.5*IQR$

$IQR=Q_{3/4}-Q_{1/4}$

練習問題

---
箱ひげ図における外れ値かどうかを判定する (外れ値でない場合に True 、外れ値の場合に False を返す) 以下の関数 isin_box を完成させる。

In [28]:
def isin_box(ndarray):
    return

解答例

---

In [29]:
def isin_box(ndarray):
    low = np.percentile(ndarray, 25)
    up = np.percentile(ndarray, 75)
    iqr = up - low
    return (low - 1.5 * iqr <= ndarray) & (ndarray <= up + 1.5 * iqr)

---

In [30]:
sample.loc[isin_box(sample['A'])]

,A,B,C,D,E
0,0.471435,-1.190976,1.432707,NaN,-0.720589
1,0.887163,0.859588,-0.636524,0.015696,NaN
...,...,...,...,...,...
998,-0.442212,-1.350463,NaN,0.842840,NaN
999,-0.592877,0.708934,1.608824,1.995282,0.033269


## 推薦図書
---
- [Python 機械学習プログラミング 達人データサイエンティストによる理論と実践](https://www.amazon.co.jp/Sebastian-Raschka/dp/4295003379/)
- [Kaggleで勝つデータ分析の技術](https://www.amazon.co.jp/Kaggle%E3%81%A7%E5%8B%9D%E3%81%A4%E3%83%87%E3%83%BC%E3%82%BF%E5%88%86%E6%9E%90%E3%81%AE%E6%8A%80%E8%A1%93-%E9%96%80%E8%84%87-%E5%A4%A7%E8%BC%94/dp/4297108437/)